In [1]:
#!pip install web3

In [2]:
#!pip install multicall

**Gearbox Protocol**

Website:        https://gearbox.fi/ <br>
dApp:           https://app.gearbox.fi/ <br>
Docs:           https://docs.gearbox.finance/ <br>
Developer Docs: https://dev.gearbox.fi/ <br>

**Gearbox Credit Accout Report** <br>
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
from gearbox_ca import (pull_abi_etherscan, parse_abi, chunks, get_data_multicall, get_logs, get_token_balance, getCreditAccountData) # see gearbox_ca.py
import os
from datetime import datetime
from pprint import pformat
import logging
import pandas as pd
from web3 import Web3
from web3.datastructures import AttributeDict
from web3.exceptions import (ContractLogicError, ABIFunctionNotFound)

######### Enable logging for debug
logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level='INFO',
        datefmt='%Y-%m-%d %H:%M:%S',
        )


Etherscan_APIKEY   = None #optional, but recommended
GearboxAddressProvider = '0xcF64698AFF7E5f27A11dff868AF228653ba53be0'

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 30}))
logging.info (f'Ethereum connected: {w3_eth.isConnected()}')


2022-11-03 11:11:56 <module> INFO Ethereum connected: True


In [9]:
logging.info('Start')
AddressProvider = Web3.toChecksumAddress(GearboxAddressProvider)
AddressProvider_abi = pull_abi_etherscan(AddressProvider)

AccountFactory = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getAccountFactory().call()
logging.info(f'AccountFactory: {AccountFactory}')
AccountFactory_abi = pull_abi_etherscan(AccountFactory)
countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccounts().call()
countCreditAccountsInStock = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccountsInStock().call()
logging.info(f'countCreditAccounts: {countCreditAccounts}')
logging.info(f'countCreditAccountsInStock: {countCreditAccountsInStock}')
masterCreditAccount = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.masterCreditAccount().call()
logging.info(f'masterCreditAccount: {masterCreditAccount}')
CreditAccount_abi = pull_abi_etherscan(masterCreditAccount)

DataCompressor = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getDataCompressor().call()
logging.info(f'DataCompressor: {DataCompressor}')
DataCompressor_abi = pull_abi_etherscan(DataCompressor)

ContractsRegister = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getContractsRegister().call()
logging.info(f'ContractsRegister: {ContractsRegister}')
ContractsRegister_abi = pull_abi_etherscan(ContractsRegister)

CreditManagers = w3_eth.eth.contract(address=ContractsRegister, abi=ContractsRegister_abi).functions.getCreditManagers().call()
logging.info(f'CreditManagers: {CreditManagers}')
version = [None] * len(CreditManagers)

cm_dict = {AccountFactory: {'token': None, 'symbol': None, 'decimals': None, 'version': None,'creditFacade': None}}
allowedTokens = {}
CreditManager_abi = Token_abi = CreditFilter_abi = creditFacade_abi = priceOracle_abi = ''

for i, CreditManager in enumerate(CreditManagers):
    logging.info(f'CreditManager {i+1} of {len(CreditManagers)}')
    
    if not CreditManager_abi:
        CreditManager_abi = pull_abi_etherscan(CreditManager)

    version[i] = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.version().call() 
    
    if i>0 and version[i]!=version[i-1]:
        CreditManager_abi = pull_abi_etherscan(CreditManager)
    
    logging.info(f'version {version[i]}')
    
    
    if version[i] == 1:
        Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlyingToken().call()  
        CreditFilter = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFilter().call()
        if not CreditFilter_abi:
            CreditFilter_abi = pull_abi_etherscan(CreditFilter) 
        allowedTokensCount = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokensCount().call()  
        priceOracle        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.priceOracle().call()
        wethAddress        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.wethAddress().call()
        CreditManager_v1_abi = CreditManager_abi
        creditFacade = None
    else:
        Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlying().call() 
        allowedTokensCount = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.collateralTokensCount().call()
        priceOracle        = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.priceOracle().call()
        wethAddress        = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.wethAddress().call()

        creditFacade = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFacade().call()
        if not creditFacade_abi:
            creditFacade_abi = pull_abi_etherscan(creditFacade)               
    
    if not Token_abi:
        Token_abi = pull_abi_etherscan(Token) 
    
    if not priceOracle_abi or version[i]!=version[i-1]:
        priceOracle_abi = pull_abi_etherscan(priceOracle)
    
    Token_symbol = w3_eth.eth.contract(address=Token, abi=Token_abi).functions.symbol().call()
    Token_decimals = w3_eth.eth.contract(address=Token, abi=Token_abi).functions.decimals().call()
    cm_dict[CreditManager] = {'version' : version[i],
                              'creditFacade': creditFacade,
                              'token': Token,
                              'symbol': Token_symbol,
                              'decimals': Token_decimals,
                              'priceOracle' : priceOracle,
                              'allowedTokensCount': allowedTokensCount,
                              'allowedTokens':{},
                             }
    for token_id in range(allowedTokensCount):
        if version[i] == 1:
            allowed_token = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokens(token_id).call()
        else:
            allowed_token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.collateralTokens(token_id).call()
            allowed_token = allowed_token[0]
            
        allowed_token_symbol = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.symbol().call()
        allowed_token_decimals = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.decimals().call()
        
        
        
        if version[i] == 1:
            #try:
            #    allowed_token_weth_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, wethAddress).call()
            #    allowed_token_weth_priceOracle = allowed_token_weth_priceOracle*(10**allowed_token_decimals)/1e18
            #except ContractLogicError:
            #    allowed_token_weth_priceOracle = None
            #    logging.info(allowed_token_symbol+'-WETH getLastPrice error')
            #try:
            #    allowed_token_underlying_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, Token).call()
            #    allowed_token_underlying_priceOracle = allowed_token_underlying_priceOracle*(10**allowed_token_decimals)/(10**Token_decimals)
            #except ContractLogicError:
            #    allowed_token_underlying_priceOracle = None
            #    logging.info(allowed_token_symbol+'-underlying getLastPrice error')
            USDC_Address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'    
            allowed_token_usd_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, USDC_Address).call()/1e18
        else:
            try:                
                #allowed_token_weth_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.convert(10**8, allowed_token, wethAddress).call()
                #allowed_token_weth_priceOracle = allowed_token_weth_priceOracle*10**(-8)
                #allowed_token_underlying_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.convert(10**8, allowed_token, Token).call()
                #allowed_token_underlying_priceOracle = allowed_token_underlying_priceOracle*10**(-8)                

                allowed_token_usd_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getPrice(allowed_token).call()
                allowed_token_usd_priceOracle=allowed_token_usd_priceOracle*10**(-8) 
            except ContractLogicError:
                #allowed_token_usd_priceOracle = allowed_token_underlying_priceOracle= None
                allowed_token_usd_priceOracle = None
                logging.info(allowed_token_symbol+'- getPrice error')            
            
        Token_dict={'symbol'          : allowed_token_symbol, 
                    'decimals'        : allowed_token_decimals,
                    'Price_USD'       : allowed_token_usd_priceOracle,
                    #'Price_WETH'      : allowed_token_weth_priceOracle,
                    #'Price_'+cm_dict[CreditManager]['symbol']: allowed_token_underlying_priceOracle,
                    }

        if allowed_token in allowedTokens:
            allowedTokens[allowed_token].update(Token_dict)
        else:
            allowedTokens[allowed_token] = Token_dict
    
df_abi = parse_abi({'AddressProvider':AddressProvider_abi, 
                    'AccountFactory': AccountFactory_abi, 
                    'DataCompressor': DataCompressor_abi,
                    'CreditAccount': CreditAccount_abi,                     
                    'CreditManager': CreditManager_abi,
                    'creditFacade': creditFacade_abi,               
                    'CreditManager_v1': CreditManager_v1_abi,
                   })

logging.info(pformat(cm_dict)) 
logging.info(pformat(allowedTokens))

2022-11-03 11:11:57 <module> INFO Start
2022-11-03 11:11:57 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-11-03 11:12:06 <module> INFO AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
2022-11-03 11:12:06 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-11-03 11:12:13 <module> INFO countCreditAccounts: 5001
2022-11-03 11:12:13 <module> INFO countCreditAccountsInStock: 4815
2022-11-03 11:12:13 <module> INFO masterCreditAccount: 0x373A292b93ff9017D28E64154ef83b99D5C4e270
2022-11-03 11:12:13 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-11-03 11:12:20 <module> INFO DataCompressor: 0x0a2CA503153Cd5CB2892a0928ac0F71F49a3c194
2022-11-03 11:12:20 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-11-03 11:12:26 <module> INFO ContractsRegister: 0xA50d4E7D8946a7c90652339CDBd262c375d54D99
2022-11-03 11:12:26 pull_abi_etherscan INFO e

2022-11-03 11:20:11 <module> INFO {'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd': {'Price_USD': 1.0010000000000001,
                                                'decimals': 2,
                                                'symbol': 'GUSD'},
 '0x06325440D014e39736583c165C2963BA99fAf14E': {'Price_USD': 1593.48035567,
                                                'decimals': 18,
                                                'symbol': 'steCRV'},
 '0x0A1D4A25d0390899b90bCD22E1Ef155003EA76d7': {'Price_USD': 1.01668105,
                                                'decimals': 18,
                                                'symbol': 'stkcvxLUSD3CRV-f'},
 '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e': {'Price_USD': 7799.1733714758975,
                                                'decimals': 18,
                                                'symbol': 'YFI'},
 '0x111111111117dC0aa78b770fA6A738034120C302': {'Price_USD': 0.5806332854518244,
                                      

In [10]:
df = pd.DataFrame()
df['id'] = range(countCreditAccounts)
for ids in list(chunks(list(df['id']), 1000)): #chunk size for multicall = 1000 (reduce in case of issues)
    id_range = list(df['id'].isin(ids))
    d_ca = get_data_multicall(w3_eth, df.loc[id_range], 'creditAccounts', df_abi[(df_abi['contract_type']=='AccountFactory')], AccountFactory)
    df.loc[id_range,'CA'] = df.loc[id_range].apply(lambda x: d_ca[x['id']], axis=1)

    d_cm = get_data_multicall(w3_eth, df.loc[id_range], 'creditManager', df_abi[(df_abi['contract_type']=='CreditAccount')])
    df.loc[id_range,'CM'] = df.loc[id_range].apply(lambda x: d_cm[x['id']], axis=1)
    
    d_amount = get_data_multicall(w3_eth, df.loc[id_range], 'borrowedAmount', df_abi[(df_abi['contract_type']=='CreditAccount')])
    df.loc[id_range,'borrowedAmount'] = df.loc[id_range].apply(lambda x: d_amount[x['id']] if d_amount[x['id']] > 1 else 0 , axis=1)
    
    d_since = get_data_multicall(w3_eth, df.loc[id_range], 'since', df_abi)
    df.loc[id_range,'Since'] = df.loc[id_range].apply(lambda x: d_since[x['id']], axis=1)

df['Since'] = df['Since'].astype(int)
df['Decimals'] = df.apply(lambda x: cm_dict[x['CM']]['decimals'], axis=1)
df['Symbol'] = df.apply(lambda x: cm_dict[x['CM']]['symbol'] , axis=1)
df['CF'] = df.apply(lambda x: cm_dict[x['CM']]['creditFacade'] , axis=1)

In [11]:
display(df)

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI,None
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH,None
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI,None
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH,None
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,None
...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None


In [16]:
#Open, Close, Repay, Liquidate
OpenCreditAccount_topic             =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
OpenCreditAccount_topic_v1          =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic                 =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic_v1              =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic            =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='RepayCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic        =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic_v1     =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateExpiredCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateExpiredCreditAccount')  & (df_abi['contract_type']=='creditFacade') &(df_abi['type']=='event')]['topic'].values[0]

logging.info(f'OpenCreditAccount_topics: {OpenCreditAccount_topic_v1}, {OpenCreditAccount_topic}' )
logging.info(f'CloseCreditAccount_topic: {CloseCreditAccount_topic_v1}, {CloseCreditAccount_topic}')
logging.info(f'RepayCreditAccount_topic: {RepayCreditAccount_topic_v1}')
logging.info(f'LiquidateCreditAccount_topic: {LiquidateCreditAccount_topic_v1}, {LiquidateCreditAccount_topic}')
logging.info(f'LiquidateExpiredCreditAccount_topic: {LiquidateExpiredCreditAccount_topic}')

logs = pd.DataFrame()
for CM in CreditManagers:
    
    if len(df.loc[df['CM']==CM]['Since'])>0:
        block_from = df.loc[df['CM']==CM]['Since'].min()
        
        CM_logs = get_logs(w3_eth, CM, df_abi,
                           [[OpenCreditAccount_topic_v1,
                             AddCollateral_topic_v1,
                             CloseCreditAccount_topic_v1,
                             RepayCreditAccount_topic_v1,
                             LiquidateCreditAccount_topic_v1,]
                             ],
                             df.loc[df['CM']==CM]['Since'].min(),
                             'latest')
        logs = logs.append(CM_logs, ignore_index = True)
        
        if cm_dict[CM]['creditFacade']:
            CF_logs = get_logs(w3_eth, cm_dict[CM]['creditFacade'], df_abi,
                               [[OpenCreditAccount_topic,
                                 AddCollateral_topic, 
                                 CloseCreditAccount_topic,
                                 LiquidateCreditAccount_topic,
                                 LiquidateExpiredCreditAccount_topic,]
                                 ],
                                 df.loc[df['CM']==CM]['Since'].min(),
                                 'latest')        
            logs = logs.append(CF_logs, ignore_index = True)

i=0
for row in df.loc[df['CM'].isin(CreditManagers)].itertuples():
    i+=1
    open_events       = logs[(logs['address'].isin([row.CM,row.CF])) & (logs['blockNumber']>=row.Since) & (logs['event']=='OpenCreditAccount')]['args'].values 
    collateral_events = logs[(logs['address'].isin([row.CM,row.CF])) & (logs['blockNumber']>=row.Since) & (logs['event']=='AddCollateral')]['args'].values 
    close_events      = logs[(logs['address'].isin([row.CM,row.CF])) & (logs['blockNumber']>=row.Since) & (logs['event']=='CloseCreditAccount')]['args'].values 
    repay_events      = logs[(logs['address'].isin([row.CM,row.CF])) & (logs['blockNumber']>=row.Since) & (logs['event']=='RepayCreditAccount')]['args'].values
    liquidate_event   = logs[(logs['address'].isin([row.CM,row.CF])) & (logs['blockNumber']>=row.Since) & (logs['event'].isin(['LiquidateCreditAccount', 'LiquidateExpiredCreditAccount']))]['args'].values

    
    
    
    CA_open_event = [x for x in open_events if x['creditAccount']== row.CA] # Open
    if len(CA_open_event) > 0:
        Borrower = CA_open_event[0]['onBehalfOf']
        df.loc[df['id']==row.id, 'Borrower'] = Borrower
        
        Collateral = sum([x['amount'] for x in CA_open_event if 'amount' in x])
        CA_collateral_event = [x for x in collateral_events if x['onBehalfOf']== Borrower] # Open
        if len(CA_collateral_event) > 0:
            Collateral = Collateral + sum(
                                            [(x['value']*(allowedTokens[x['token']]['Price_USD'])*(10**-allowedTokens[x['token']]['decimals']))
                                             /(allowedTokens[cm_dict[row.CM]['token']]['Price_USD']*(10**-row.Decimals))
                                            for x in CA_collateral_event 
                                            if allowedTokens[cm_dict[row.CM]['token']]['Price_USD']>0
                                            ]
                                         )
        df.loc[df['id']==row.id, 'Collateral'] = Collateral
                           
        CA_close_event = [x for x in close_events if x['to']== Borrower] # Close
        if len(CA_close_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]

        CA_repay_event = [x for x in repay_events if x['to']== Borrower] # Repay
        if len(CA_repay_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]

        CA_liquidate_event = [x for x in liquidate_event if ('owner' in x and x['owner']==Borrower) or ('borrower' in x and x['borrower']==Borrower)] # Liquidate
        if len(CA_liquidate_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]
            
    if i % 50 == 0:
        logging.info (i)

logging.info(f'{i} end')


2022-11-03 11:27:01 <module> INFO OpenCreditAccount_topics: 0x7b20ae77867a263a1074203a2da261ef0d096c99395c59c9d4a0104b9f334a27, 0xfa2baf5d3eb95569f312f22477b246f9d4c50276f1cb3ded8e1aeadcbc07a763
2022-11-03 11:27:01 <module> INFO CloseCreditAccount_topic: 0xca05b632388199c23de1352b2e96fd72a0ec71611683330b38060c004bbf0a76, 0x460ad03b1cf79b1d64d3aefa28475f110ab66e84649c52bb41ed796b9b391981
2022-11-03 11:27:01 <module> INFO RepayCreditAccount_topic: 0xe7c7987373a0cc4913d307f23ab8ef02e0333a2af445065e2ef7636cffc6daa7
2022-11-03 11:27:01 <module> INFO LiquidateCreditAccount_topic: 0x5e5da6c348e62989f9cfe029252433fc99009b7d28fa3c20d675520a10ff5896, 0x7dfecd8419723a9d3954585a30c2a270165d70aafa146c11c1e1b88ae1439064
2022-11-03 11:27:01 <module> INFO LiquidateExpiredCreditAccount_topic: 0xfada13ff2d5e1a1d9da37ad4a4130893b34e3c69c32b17f87ec31fd661b86707
2022-11-03 11:27:11 <module> INFO 50
2022-11-03 11:27:11 <module> INFO 100
2022-11-03 11:27:11 <module> INFO 150
2022-11-03 11:27:12 <module> INFO

In [17]:
display(df)

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,None,0.000000e+00
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,None,0.000000e+00
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,None,0.000000e+00
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,None,0.000000e+00
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,None,0x53829d517D8fA7D59d3D40E20251e519C079985F,3.000000e+17
...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,NaN


In [18]:
data_cols = [x['name'] for x in df_abi[df_abi['name']=='getCreditAccountData']['abi'].values[0]['outputs'][0]['components']]

batchtime = datetime.utcnow()
df['batchtime'] = batchtime

for ids in list(chunks(list(df[pd.notna(df['Borrower'])].loc[:,'id']), 1000)): #chunk size for multicall = 1000 (reduce in case of issues)
    id_range = list(df['id'].isin(ids))
    try:
        d_data = get_data_multicall(w3_eth, df.loc[id_range], 'getCreditAccountData', df_abi, DataCompressor)
    except ContractLogicError:
        logging.error('getCreditAccountData: Multicall error, calling it one by one')
        d_data = {x:getCreditAccountData(x,y,z) for x,y,z in zip(df.loc[id_range]['id'],df.loc[id_range]['CM'],df.loc[id_range]['Borrower'])}
        d_data = AttributeDict.recursive(d_data)
    for data in data_cols:
        if data not in ['balances', 'inUse', 'borrower', 'addr', 'borrower', 'creditManager', 'since']: #duplicate columns
            df.loc[id_range, data] = df.loc[id_range].apply(lambda x: {y:z for y, z in zip(data_cols, d_data[x['id']])}[data] if d_data[x['id']] else None
                                                            , axis=1)
        
    tokens_to_frame = [allowedTokens[x]['symbol'] for x in allowedTokens] 
    for token in tokens_to_frame:
        df.loc[id_range, 'Balance_'+token] = df.loc[id_range].apply(lambda x: get_token_balance(x, token, data_cols, d_data, allowedTokens)
                                                        , axis=1)


In [19]:
display(df) 

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral,...,Balance_FXS,Balance_LQTY,Balance_stkcvxFRAX3CRV-f,Balance_stkcvxLUSD3CRV-f,Balance_stkcvxgusd3CRV,Balance_stkcvxcrvPlain3andSUSD,Balance_stkcvx3Crv,Balance_stkcvxcrvFRAX,Balance_stkcvxsteCRV,Balance_wstETH
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,None,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,None,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,None,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,None,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,None,0x53829d517D8fA7D59d3D40E20251e519C079985F,3.000000e+17,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
display(df.columns)

Index(['id', 'CA', 'CM', 'borrowedAmount', 'Since', 'Decimals', 'Symbol', 'CF',
       'Borrower', 'Collateral', 'batchtime', 'underlying',
       'borrowedAmountPlusInterest', 'borrowedAmountPlusInterestAndFees',
       'totalValue', 'healthFactor', 'borrowRate', 'repayAmount',
       'liquidationAmount', 'canBeClosed', 'cumulativeIndexAtOpen', 'version',
       'enabledTokenMask', 'Balance_DAI', 'Balance_1INCH', 'Balance_AAVE',
       'Balance_COMP', 'Balance_DPI', 'Balance_FEI', 'Balance_LINK',
       'Balance_SNX', 'Balance_UNI', 'Balance_USDC', 'Balance_WBTC',
       'Balance_WETH', 'Balance_YFI', 'Balance_yvDAI', 'Balance_yvUSDC',
       'Balance_CRV', 'Balance_SUSHI', 'Balance_LDO', 'Balance_FTM',
       'Balance_LUNA', 'Balance_G-OBS', 'Balance_G-BLOCK', 'Balance_stETH',
       'Balance_USDT', 'Balance_sUSD', 'Balance_FRAX', 'Balance_GUSD',
       'Balance_LUSD', 'Balance_steCRV', 'Balance_cvxsteCRV', 'Balance_3Crv',
       'Balance_cvx3Crv', 'Balance_FRAX3CRV-f', 'Balance_cvxF

In [21]:
display(df[pd.notna(df['Borrower'])]) # active CAs

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,CF,Borrower,Collateral,...,Balance_FXS,Balance_LQTY,Balance_stkcvxFRAX3CRV-f,Balance_stkcvxLUSD3CRV-f,Balance_stkcvxgusd3CRV,Balance_stkcvxcrvPlain3andSUSD,Balance_stkcvx3Crv,Balance_stkcvxcrvFRAX,Balance_stkcvxsteCRV,Balance_wstETH
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,None,0x53829d517D8fA7D59d3D40E20251e519C079985F,3.000000e+17,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
8,8,0xF91027C04807c71c7771a2314A622dFA0fE509A1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13864789,18.0,WETH,None,0x896b94f4f27f12369698C302e2049cAe86936BbB,1.000000e+18,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
11,11,0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,290000000000000000,13864924,18.0,WETH,None,0xC16414AC1fedfDAC4F8A09674D994e1BbB9d7113,1.000000e+18,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
14,14,0x654C0aDcA189042B16A849d2fF2ED476F525c39e,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,15000000000000000000000,13865587,18.0,DAI,None,0x3aB0818Ba16175D5a41a4076De227c8a55AD6038,1.000000e+22,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
21,21,0xBababB47c2A3543a13719a4BfA5a2cc03d1D1936,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,350000000000000000,13867661,18.0,WETH,None,0xC045aA09a487B68F5895e1Ba6d867384b9710670,3.500000e+17,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,968,0x3f9912251238e6bD4f622E8a59701c3fE1AE4162,0x95357303f995e184A7998dA6C6eA35cC728A1900,115200000000,15883017,6.0,USDC,0x61fbb350e39cc7bF22C01A469cf03085774184aa,0xe4fb9ca52E39B83BE69E74E7Fac8f807A9c16eF9,1.920000e+10,...,0.0,0.0,0,65527435050898687215579,0.0,0.0,0.0,0,0,0.0
969,969,0x2A478674cD1ecF72DA17C8248c16200c16d98C24,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,76590000000000000000,15883510,18.0,WETH,0xC59135f449bb623501145443c70A30eE648Fa304,0x83f3E65150a606E9f4450b2C61DcbD6aacFC80c7,1.850000e+19,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
970,970,0x2424cF88Ebf5C6CB9138aCb7fe9576B5854ecA85,0x95357303f995e184A7998dA6C6eA35cC728A1900,1000000000000,15884101,6.0,USDC,0x61fbb350e39cc7bF22C01A469cf03085774184aa,0x87839e0378c62d8962C76726cfdd932a97eF626a,2.000000e+11,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0
971,971,0x0a04F1c6ee8642856DD4CcFD46e20aeD19c76c2C,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,599950000000000000000,15884226,18.0,WETH,0xC59135f449bb623501145443c70A30eE648Fa304,0x69c509e9765c49CC6B9B4568b90Aa47B89F4992f,1.000000e+20,...,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0


In [22]:
logging.info(f'batchtime {batchtime}' )
logging.info(f'countCreditAccounts - countCreditAccountsInStock = {countCreditAccounts - countCreditAccountsInStock}')

2022-11-03 11:28:27 <module> INFO batchtime 2022-11-02 22:27:59.572680
2022-11-03 11:28:27 <module> INFO countCreditAccounts - countCreditAccountsInStock = 186


In [23]:
df_price_oracle = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_price_oracle = df_price_oracle.set_index('symbol', drop=True).transpose()
df_price_oracle.columns = ['Price_'+x for x in df_price_oracle.columns]
df_price_oracle[[x.replace('Price','Decimals') for x in df_price_oracle.columns]] = df_price_oracle.loc['decimals',:]
df_price_oracle = df_price_oracle.drop(index = 'decimals')

df_price_oracle = df_price_oracle.reset_index()
df_price_oracle = df_price_oracle.rename(columns={'index': 'Price_Asset'})
df_price_oracle['Price_Asset'] = df_price_oracle['Price_Asset'].apply(lambda x: x.replace('Price_',''))

df_price_oracle['batchtime'] = batchtime


In [24]:
display(df_price_oracle)

,Price_Asset,Price_DAI,Price_1INCH,Price_AAVE,Price_COMP,Price_DPI,Price_FEI,Price_LINK,Price_SNX,Price_UNI,...,Decimals_LQTY,Decimals_stkcvxFRAX3CRV-f,Decimals_stkcvxLUSD3CRV-f,Decimals_stkcvxgusd3CRV,Decimals_stkcvxcrvPlain3andSUSD,Decimals_stkcvx3Crv,Decimals_stkcvxcrvFRAX,Decimals_stkcvxsteCRV,Decimals_wstETH,batchtime
0,USD,1.0,0.580633,79.622446,46.203808,84.359216,0.994339,7.39505,2.351,6.945405,...,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,2022-11-02 22:27:59.572680


In [27]:
df_token_price = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_token_price.columns = [x.replace('Price_','') for x in df_token_price.columns]
df_token_price['batchtime'] = batchtime
df_token_price = df_token_price.rename(columns = {'symbol':'token'})
numeric_cols = [x for x in df_token_price.columns if x not in ['token', 'batchtime','decimals']]
df_token_price[numeric_cols] = df_token_price[numeric_cols]
df_token_price[numeric_cols] = df_token_price[numeric_cols].astype('float64')

In [28]:
display(df_token_price)

,token,decimals,USD,batchtime
0,DAI,18,1.000000,2022-11-02 22:27:59.572680
1,1INCH,18,0.580633,2022-11-02 22:27:59.572680
2,AAVE,18,79.622446,2022-11-02 22:27:59.572680
3,COMP,18,46.203808,2022-11-02 22:27:59.572680
4,DPI,18,84.359216,2022-11-02 22:27:59.572680
5,FEI,18,0.994339,2022-11-02 22:27:59.572680
6,LINK,18,7.395050,2022-11-02 22:27:59.572680
7,SNX,18,2.351000,2022-11-02 22:27:59.572680
8,UNI,18,6.945405,2022-11-02 22:27:59.572680
9,USDC,6,1.000023,2022-11-02 22:27:59.572680


In [ ]:
#Open, Close, Repay, Liquidate
OpenCreditAccount_topic             =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
OpenCreditAccount_topic_v1          =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic                 =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic_v1              =  df_abi[(df_abi['name']=='AddCollateral') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic            =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='CloseCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic_v1         =  df_abi[(df_abi['name']=='RepayCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic        =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='creditFacade') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic_v1     =  df_abi[(df_abi['name']=='LiquidateCreditAccount') & (df_abi['contract_type']=='CreditManager_v1') & (df_abi['type']=='event')]['topic'].values[0]
LiquidateExpiredCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateExpiredCreditAccount')  & (df_abi['contract_type']=='creditFacade') &(df_abi['type']=='event')]['topic'].values[0]

from_block = 0

logging.info(f'from_block={from_block}')

df_events = pd.DataFrame()

for CM in CreditManagers:
    logging.info(f'get_logs from CM {CM}')    
    logs_v1 = get_logs(w3_eth, CM, df_abi, 
                    [[OpenCreditAccount_topic_v1,
                      AddCollateral_topic_v1,
                      CloseCreditAccount_topic_v1,
                      RepayCreditAccount_topic_v1,
                      LiquidateCreditAccount_topic_v1,]
                    ],
                    from_block,
                    'latest')
    df_events = df_events.append(logs_v1, ignore_index = True)  
        
    if cm_dict[CM]['creditFacade']:
        logging.info(f'get_logs from CF {cm_dict[CM]["creditFacade"]}') 
        logs = get_logs(w3_eth, cm_dict[CM]['creditFacade'], df_abi, 
                        [[OpenCreditAccount_topic,
                          AddCollateral_topic, 
                          CloseCreditAccount_topic, 
                          LiquidateCreditAccount_topic, 
                          LiquidateExpiredCreditAccount_topic]
                        ],
                        from_block,
                        'latest')
        df_events = df_events.append(logs, ignore_index = True)  
        
logging.info(f'number of events={len(df_events)}')
cf_dict = {cm_dict[y]['creditFacade']:{'CreditManager':y} for y in cm_dict if cm_dict[y]['creditFacade']}

if len(df_events)>0:
    df_events['CF'] = df_events['address'].apply(lambda x: x if x in cf_dict else None)
    df_events['CM'] = df_events['address'].apply(lambda x: x if x in cm_dict else cf_dict[x]['CreditManager'])
    
    
    df_events['CM_Token'] = df_events['CM'].apply(lambda x: cm_dict[x]['symbol'])
    df_events['blockHash'] = df_events['blockHash'].apply(lambda x: x.hex())
    df_events['transactionHash'] = df_events['transactionHash'].apply(lambda x: x.hex())
    df_events['Borrower'] = df_events.apply(lambda x: x['args']['to'] 
                                                  if x['event'] in ['CloseCreditAccount','CloseCreditAccount'] 
                                                  else x['args']['onBehalfOf'] if x['event'] in ['OpenCreditAccount','AddCollateral']
                                                  else x['args']['borrower'] if x['event'] in ['IncreaseBorrowedAmount']
                                                  else x['args']['owner'] if 'owner' in x['args'] and x['event'] in ['LiquidateCreditAccount', 'LiquidateExpiredCreditAccount']
                                                  else x['args']['borrower'] if 'borrower' in x['args'] and x['event'] in ['LiquidateCreditAccount', 'LiquidateExpiredCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Liquidator'] = df_events.apply(lambda x: x['args']['liquidator'] if x['event'] in ['LiquidateCreditAccount','LiquidateExpiredCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Collateral_Token'] = df_events.apply(lambda x: allowedTokens[x['args']['token']]['symbol'] if x['event']=='AddCollateral'
                                                    else cm_dict[x['CM']]['symbol']
                                        ,axis=1)
    df_events['Collateral'] = df_events.apply(lambda x: x['args']['amount']*(10**-cm_dict[x['CM']]['decimals']) if x['event']=='OpenCreditAccount' and 'amount' in x['args']
                                                    else x['args']['value']*(10**-allowedTokens[x['args']['token']]['decimals']) if x['event']=='AddCollateral'
                                                    else None
                                        ,axis=1)

    logging.info('pull event timestamp begin')
    df_events['blockTimestamp'] = df_events['blockNumber'].apply(lambda x: datetime.utcfromtimestamp(w3_eth.eth.getBlock(x)['timestamp']))
    logging.info('pull event timestamp end')
    
    logging.info('pull transaction fee begin')
    df_events['transaction_receipt'] = df_events['transactionHash'].apply(lambda x: dict(w3_eth.eth.get_transaction_receipt(x)))
    df_events['transaction_fee'] = df_events['transaction_receipt'].apply(lambda x: x['gasUsed']*x['effectiveGasPrice']/1e18)
    logging.info('pull transaction fee end')    
    
    #df_events[['args','transaction_receipt']] = df_events[['args','transaction_receipt']].apply(str)
    #df_events['Liquidator'] = df_events['Liquidator'].astype('object')    
    #df_events['Collateral'] = df_events['Collateral'].astype('float64')
    df_events.drop(columns = 'logIndex')
    df_events = df_events.sort_values('blockTimestamp', ignore_index = True)

else:
    logging.info(f'No new events since block {from_block}')
    

In [74]:
display(df_events)

,address,args,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex,CF,CM,CM_Token,Borrower,Liquidator,Collateral_Token,Collateral,blockTimestamp,transaction_receipt,transaction_fee
0,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,"(sender, onBehalfOf, creditAccount, amount, bo...",0x6191b1dc0938794510eb96bc4e9b1ec8d1a91fa9352b...,13858003,OpenCreditAccount,316,0xc322fdd8d731a58b3e95414d3201fa53be82fd780e82...,220,None,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,DAI,0x8a4B89D76A1a745a4A1aDEBd3793253FBa0adADc,None,DAI,1000.000000,2021-12-22 23:40:25,{'blockHash': b'a\x91\xb1\xdc\t8yE\x10\xeb\x96...,0.029386
1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"(sender, onBehalfOf, creditAccount, amount, bo...",0x880436c3d9131fc9a17a6d79ad2acdcd50974d1cde97...,13862947,OpenCreditAccount,458,0x37aa66296c27993bdc5e81e27c6d819e7466df0c070c...,356,None,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,1.250000,2021-12-23 18:09:38,{'blockHash': b'\x88\x046\xc3\xd9\x13\x1f\xc9\...,0.047534
2,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,"(sender, onBehalfOf, creditAccount, amount, bo...",0xef59810b1015fa4f1a64d68a6166fe9d23e40cb1f585...,13862953,OpenCreditAccount,168,0xae23bb735281dbfede5cc3352dad8b7bdaa529b27989...,125,None,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,DAI,0x7BAFC0D5c5892f2041FD9F2415A7611042218e22,None,DAI,1000.000000,2021-12-23 18:11:05,{'blockHash': b'\xefY\x81\x0b\x10\x15\xfaO\x1a...,0.036783
3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"(onBehalfOf, token, value)",0x60a3c671e4f5d06281611a576410de04c4ec1478971b...,13862954,AddCollateral,434,0xf37a9984138c6b9c02bc657593ef6d0680dd69e2f002...,312,None,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,0.550000,2021-12-23 18:11:07,{'blockHash': b'`\xa3\xc6q\xe4\xf5\xd0b\x81a\x...,0.011669
4,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"(owner, to, remainingFunds)",0x6db00f3e2855ea71eb99701dfd5adb413666e13b2e72...,13862971,CloseCreditAccount,300,0x380f4f2acff142b5a407b51cabbd50b611d9702aa99b...,160,None,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,NaN,2021-12-23 18:14:12,{'blockHash': b'm\xb0\x0f>(U\xeaq\xeb\x99p\x1d...,0.029957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,0xf6f4F24ae50206A07B8B32629AeB6cb1837d854F,"(onBehalfOf, token, value)",0x53941bf81dbbbb544440cdcdcb8811bbb0f19a65c776...,15871940,AddCollateral,313,0xc0b2b81dbfb41547aed88e60233ee0587cd883128b48...,128,0xf6f4F24ae50206A07B8B32629AeB6cb1837d854F,0x672461Bfc20DD783444a830Ad4c38b345aB6E2f7,DAI,0xf5B4635f3858a2Bc80a17BD4450E05a35Ad99A72,None,DAI,49452.643702,2022-11-01 01:32:11,{'blockHash': b'S\x94\x1b\xf8\x1d\xbb\xbbTD@\x...,0.017856
2270,0x61fbb350e39cc7bF22C01A469cf03085774184aa,"(borrower, to)",0xfd1c37ad32604a74a6a9c2ad6c1382afbd7cce2f4498...,15872322,CloseCreditAccount,167,0x1276ca34838b12c1b53bcf53705f872a7d8ff5a0610a...,65,0x61fbb350e39cc7bF22C01A469cf03085774184aa,0x95357303f995e184A7998dA6C6eA35cC728A1900,USDC,0x44f12eF573F5A70464B7904E037aff4eC0061c6F,None,USDC,NaN,2022-11-01 02:49:35,{'blockHash': b'\xfd\x1c7\xad2`Jt\xa6\xa9\xc2\...,0.002844
2271,0x61fbb350e39cc7bF22C01A469cf03085774184aa,"(onBehalfOf, token, value)",0x7bad3c4c3d0377c6906a5dfcbaac33a147412a40d4a0...,15872349,AddCollateral,386,0x9df77a760e6cc178e8838681a9f7cedcf647e014edd7...,154,0x61fbb350e39cc7bF22C01A469cf03085774184aa,0x95357303f995e184A7998dA6C6eA35cC728A1900,USDC,0xD3474BD2f558B12e83810EC7d8aF14f2d1D08137,None,USDC,53147.920687,2022-11-01 02:54:59,{'blockHash': b'{\xad<L=\x03w\xc6\x90j]\xfc\xb...,0.000857
2272,0xC59135f449bb623501145443c70A30eE648Fa304,"(onBehalfOf, creditAccount, borrowAmount, refe...",0x550df1b0d8475e5a3f455692a7b3d7cf17c990489b4e...,15872863,OpenCreditAccount,264,0x49fe0249064fbc035b04a3cb8d6d862dc107bdbe15f0...,118,0xC59135f449bb623501145443c70A30eE648Fa304,0x5887ad4Cb2352E7F01527035fAa3AE0Ef2cE2b9B,WETH,0xCbD6b0

# Data Studio Report
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0

In [ ]:
CF_logs = get_logs(w3_eth, '0xC59135f449bb623501145443c70A30eE648Fa304', df_abi,
                                   [[OpenCreditAccount_topic,
                                     AddCollateral_topic,
                                     CloseCreditAccount_topic,
                                     LiquidateCreditAccount_topic,
                                     LiquidateExpiredCreditAccount_topic,]
                                     ],
                                     df.loc[df['CM']==CM]['Since'].min(),
                                     'latest')

In [14]:
[x for x in liquidate_event] # if x['owner']== Borrower] 

[AttributeDict({'borrower': '0x39c78cE4fBd59722f48A3271fE11F9AC5F933Ba0',
  'liquidator': '0x6C6B87D44d239B3750bf9BAdce26a9a0A3d2364e',
  'to': '0x6C6B87D44d239B3750bf9BAdce26a9a0A3d2364e',
  'remainingFunds': 11081476085457680684})]